### This notebook has been run multiple times to ensuret the results in the paper

In [1]:
from datasets import load_dataset


dataset = load_dataset("csv", data_files="challenge_data/clinc150/full_larson_orig_train.csv", split='train', keep_default_na=False)

dct_dataset = dataset### This notebook has been run multiple times to ensuret the results in the paper.train_test_split(test_size=0.01) # this changes based on the dataset

dataset_train = dct_dataset['train']
dataset_test = dct_dataset['test']

/opt/anacondas/anaconda3/envs/EMNLP/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/jan_cegin/.cache/huggingface/datasets/csv/default-cebb2c7cef2b1b4f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset_train.map(tokenize_function, batched=True)
tokenized_test_datasets = dataset_test.map(tokenize_function, batched=True)

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=40)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="bert_taboo")

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="bert_taboo", evaluation_strategy="no", save_strategy="no", per_device_eval_batch_size=32, per_device_train_batch_size=4, learning_rate=1e-5, num_train_epochs=5, save_steps=3000)

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anacondas/anaconda3/envs/EMNLP/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3465
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4335
  Number of trainable parameters = 335182888


Step,Training Loss


In [ ]:
#trainer.evaluate(tokenized_test_datasets)

#### evaluating on different data

In [ ]:
dataset_orig_test = load_dataset("csv", data_files="challenge_data/clinc150/full_larson_orig_test.csv", split='train', keep_default_na=False)


dataset = load_dataset("csv", data_files="challenge_data/clinc150/full_larson_human.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_human_test = dct_dataset['test']


dataset = load_dataset("csv", data_files="challenge_data/clinc150/full_larson_gpt.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_gpt_test = dct_dataset['test']



dataset = load_dataset("csv", data_files="challenge_data/clinc150/full_larson_gpt_filt.csv", split='train', keep_default_na=False)

dct_dataset = dataset.train_test_split(test_size=0.2)

dataset_gpt_test_filt = dct_dataset['test']

In [ ]:
tokenized_dataset_orig_t = dataset_orig_test.map(tokenize_function, batched=True)
tokenized_dataset_h_t= dataset_human_test.map(tokenize_function, batched=True)
tokenized_dataset_gpt = dataset_gpt_test.map(tokenize_function, batched=True)
tokenized_dataset_gpt_filt = dataset_gpt_test_filt.map(tokenize_function, batched=True)

In [15]:
trainer.evaluate(tokenized_dataset_orig_t)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 969
  Batch size = 32


{'eval_loss': 0.1414939910173416,
 'eval_accuracy': 0.9628482972136223,
 'eval_runtime': 21.5505,
 'eval_samples_per_second': 44.964,
 'eval_steps_per_second': 1.438,
 'epoch': 5.0}

In [16]:
trainer.evaluate(tokenized_dataset_h_t)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 604
  Batch size = 32


{'eval_loss': 0.7707468867301941,
 'eval_accuracy': 0.8410596026490066,
 'eval_runtime': 28.5,
 'eval_samples_per_second': 21.193,
 'eval_steps_per_second': 0.667,
 'epoch': 5.0}

In [17]:
trainer.evaluate(tokenized_dataset_gpt)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1889
  Batch size = 32


{'eval_loss': 1.5607881546020508,
 'eval_accuracy': 0.7517204870301747,
 'eval_runtime': 89.2222,
 'eval_samples_per_second': 21.172,
 'eval_steps_per_second': 0.672,
 'epoch': 5.0}

In [18]:
trainer.evaluate(tokenized_dataset_gpt_filt)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1741
  Batch size = 32


{'eval_loss': 1.6140258312225342,
 'eval_accuracy': 0.7501435956346927,
 'eval_runtime': 82.3096,
 'eval_samples_per_second': 21.152,
 'eval_steps_per_second': 0.668,
 'epoch': 5.0}